In [37]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import unicodedata
from tkinter import filedialog
import zipfile
import os
import requests
import dask.dataframe as dd

# Função para remover acentos e caracteres especiais
def remove_special_characters(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

# Função para realizar o JOIN entre os DataFrames
def perform_join_and_display_ages():
    # Solicitar a URL do arquivo CSV da Bahia
    url_bahia = bahia_url_entry.get()
    if not url_bahia:
        result_label.config(text="URL da Bahia não fornecida.")
        return

    # Caminho para o arquivo ZIP do Rio de Janeiro
    zip_file_path = "C:\\Users\\Eduarda\\OneDrive\\Área de Trabalho\\PIBITI-IFBA-CNPq-2023-2024\\Desafio 1\\ba1.zip"

    # Pasta de destino para extrair os arquivos
    extracted_folder = "C:\\Users\\Eduarda\\OneDrive\\Área de Trabalho\\PIBITI-IFBA-CNPq-2023-2024\\Desafio 1\\extracted_files"

    # Verificar se a pasta de destino existe e, se não existir, criar
    if not os.path.exists(extracted_folder):
        os.makedirs(extracted_folder)

    # Extrair os arquivos do arquivo ZIP
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder)

    # Caminho para o arquivo CSV extraído
    csv_file_path = os.path.join(extracted_folder, 'ba1.csv')

    # Especificar tipos de dados para colunas problemáticas
    column_types = {'col12': str, 'col26': int, 'col31': float}

    try:
        # Ler o arquivo CSV do Bahia 2 com especificação de tipos de dados
        df_rj = pd.read_csv(csv_file_path, delimiter=determinar_delimitador(csv_file_path), dtype=column_types, low_memory=False)
    except Exception as e:
        error_message = f"Erro ao ler o arquivo CSV do Bahia 2. Detalhes do erro: {str(e)}"
        result_label.config(text=error_message)
        print(error_message)
        return

    # Download do arquivo CSV da Bahia
    try:
        response = requests.get(url_bahia)
        with open('bahia.csv', 'wb') as file:
            file.write(response.content)
    except Exception as e:
        error_message = f"Erro ao fazer o download do arquivo CSV da Bahia. Detalhes do erro: {str(e)}"
        result_label.config(text=error_message)
        print(error_message)
        return

    # Ler o arquivo CSV da Bahia após o download usando Dask
    try:
        df_bahia = dd.read_csv('bahia.csv', delimiter=determinar_delimitador('bahia.csv'))
    except Exception as e:
        error_message = f"Erro ao ler o arquivo CSV da Bahia. Detalhes do erro: {str(e)}"
        result_label.config(text=error_message)
        print(error_message)
        return

    # Realizar o merge usando Dask
    join_type = join_type_var.get()  # Tipo de JOIN selecionado (inner, outer, left, right)
    merged_df = dd.merge(df_bahia, df_rj, on='sintomas', how=join_type)

    # Exibir a coluna 'idade' após o merge
    result_label.config(text="Coluna 'idade' após o JOIN:\n" + str(merged_df['idade'].compute()))

# Função para determinar o delimitador do arquivo CSV
def determinar_delimitador(url):
    with open(url, 'r') as file:
        line = file.readline()
        delimiters = [',', ';', '\t']
        for delimiter in delimiters:
            if delimiter in line:
                return delimiter
    return ','  # Delimitador padrão é a vírgula

# Configuração da janela principal
root = tk.Tk()
root.title("Operação de JOIN e Exibição de Idades")

# URL da Bahia
bahia_url_label = ttk.Label(root, text="URL do arquivo CSV da Bahia:")
bahia_url_entry = ttk.Entry(root, width=40)

# Tipo de JOIN
join_type_var = tk.StringVar()
join_type_label = ttk.Label(root, text="Tipo de JOIN:")
join_type_combobox = ttk.Combobox(root, textvariable=join_type_var, values=["inner", "outer", "left", "right"])

# Junção Interna (Inner Join):
    # Retorna apenas as linhas que têm correspondências em ambos os DataFrames.
    # Exclui as linhas sem correspondências.
# Junção Externa (Outer Join):
    # Retorna todas as linhas de ambos os DataFrames.
    # Preenche com valores nulos onde não há correspondências.
# Junção à Esquerda (Left Join):
    # Retorna todas as linhas do DataFrame da esquerda e as correspondentes do DataFrame da direita.
    # Preenche com valores nulos onde não há correspondências no DataFrame da direita.
# Junção à Direita (Right Join): 
    # É semelhante ao Left Join, mas retorna todas as linhas do DataFrame da direita e as correspondentes do DataFrame da esquerda.
    # Preenche com valores nulos onde não há correspondências no DataFrame da esquerda.

# Botão para realizar a operação de JOIN
join_button = ttk.Button(root, text="Realizar JOIN e Exibir Idades", command=perform_join_and_display_ages)

# Rótulo para exibir a coluna 'IDADES' após o merge
result_label = ttk.Label(root, text="")

# Posicionamento dos widgets
bahia_url_label.grid(row=0, column=0, sticky="w")
bahia_url_entry.grid(row=0, column=1, columnspan=2, sticky="w")
join_type_label.grid(row=1, column=0, sticky="w")
join_type_combobox.grid(row=1, column=1, columnspan=2, sticky="w")
join_button.grid(row=2, column=0, columnspan=3, pady=10)
result_label.grid(row=3, column=0, columnspan=3, pady=10)

# Iniciar a interface gráfica
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Users\Eduarda\anaconda3\Lib\site-packages\pandas\core\indexes\base.py", line 3653, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'idade'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "d:\Users\Eduarda\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Eduarda\AppData\Local\Temp\ipykernel_13108\3707019221.py", line